First we split the dataset. This only needs to be done once. We use a seed to ensure consistent results.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path
import time

from utils.pytorch_models import ResNet18
from models.poolformer import create_poolformer_s12
from models.ConvMixer import create_convmixer_1024_20
from models.MLPMixer import create_mlp_mixer
from utils.clients import GlobalClient
from utils.pytorch_utils import start_cuda

In [2]:
c1_path = Path("data/c1_train.csv")
c2_path = Path("data/c2_train.csv")
c3_path = Path("data/c3_train.csv")
if not (c1_path.exists() and c2_path.exists() and c3_path.exists()):
    df = pd.read_csv("data/train.csv", header=None)

    seed = 42

    df_c1, temp = train_test_split(df, test_size=(2/3), random_state=seed)
    df_c2, df_c3 = train_test_split(temp, test_size=0.5, random_state=seed)

    df_c1.to_csv("data/c1_train.csv", index=False, header=None)
    df_c2.to_csv("data/c2_train.csv", index=False, header=None)
    df_c3.to_csv("data/c3_train.csv", index=False, header=None)

In [3]:
cuda_no = 1
batch_size = 128
num_workers = 0
epochs = 10
communication_rounds = 20

channels = 10
num_classes = 19
dataset_filter = "serbia"

## Train ConvMixer

In [4]:
convmixer = create_convmixer_1024_20(channels, num_classes, pretrained=False)
global_client_convmixer = GlobalClient(
    model=convmixer,
    lmdb_path="data/BigEarth_Serbia_Summer_S2.lmdb",
    val_path="data/test.csv",
    csv_paths=["data/c1_train.csv", "data/c2_train.csv", "data/c3_train.csv"],
)

In [5]:
global_convmixer_results, global_convmixer_client_results = global_client_convmixer.train(communication_rounds=communication_rounds, epochs=epochs)


Round 1/20
----------
Epoch 1/10
----------


training: 100%|██████████| 21/21 [00:41<00:00,  1.98s/it]


Epoch 2/10
----------


training: 100%|██████████| 21/21 [00:09<00:00,  2.24it/s]


Epoch 3/10
----------


...

Epoch 9/10
----------


training: 100%|██████████| 21/21 [00:08<00:00,  2.35it/s]


Epoch 10/10
----------


test: 100%|██████████| 28/28 [00:14<00:00,  1.95it/s]
/root/micromamba/envs/cv4rs/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/micromamba/envs/cv4rs/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/micromamba/envs/cv4rs/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:980: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/root/micromamba/envs/cv4rs/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:980: UserWarnin

micro     precision: 0.8004 | recall: 0.6388 | f1-score: 0.7105 | support: 9725 | mAP: 0.8005
macro     precision: 0.4457 | recall: 0.2899 | f1-score: 0.3321 | support: 9725 | mAP: 0.3826



## Train PoolFormer

In [ ]:
poolformer_s12 = create_poolformer_s12(in_chans=channels, num_classes=num_classes)
global_client_poolformer = GlobalClient(
    model=poolformer_s12,
    lmdb_path="data/BigEarth_Serbia_Summer_S2.lmdb",
    val_path="data/test.csv",
    csv_paths=["data/c1_train.csv", "data/c2_train.csv", "data/c3_train.csv"],
)

In [ ]:
global_poolformer_results, global_poolformer_client_results = global_client_poolformer.train(communication_rounds=communication_rounds, epochs=epochs)

## Train ResNet18

In [4]:
resnet18 = ResNet18(num_cls=num_classes, channels=channels, pretrained=False)
global_client_resnet18 = GlobalClient(
    model=resnet18,
    lmdb_path="data/BigEarth_Serbia_Summer_S2.lmdb",
    val_path="data/test.csv",
    csv_paths=["data/c1_train.csv", "data/c2_train.csv", "data/c3_train.csv"],
)

/root/micromamba/envs/cv4rs/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/micromamba/envs/cv4rs/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
global_resnet18_results, global_resnet18_client_results = global_client_resnet18.train(communication_rounds=communication_rounds, epochs=epochs)

Round 1/20
----------
Epoch 1/10
----------


training:   0%|          | 0/21 [00:00<?, ?it/s]

training: 100%|██████████| 21/21 [00:46<00:00,  2.20s/it]


Epoch 2/10
----------


training: 100%|██████████| 21/21 [00:09<00:00,  2.33it/s]


...

training: 100%|██████████| 21/21 [00:08<00:00,  2.56it/s]


Epoch 10/10
----------


test: 100%|██████████| 28/28 [00:12<00:00,  2.25it/s]
/root/micromamba/envs/cv4rs/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/micromamba/envs/cv4rs/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/micromamba/envs/cv4rs/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, m

micro     precision: 0.7955 | recall: 0.7237 | f1-score: 0.7579 | support: 9725 | mAP: 0.8394
macro     precision: 0.4740 | recall: 0.3629 | f1-score: 0.3956 | support: 9725 | mAP: 0.4395




/root/micromamba/envs/cv4rs/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/micromamba/envs/cv4rs/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/micromamba/envs/cv4rs/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:980: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/root/micromamba/envs/cv4rs/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:980: UserWarning: No positive class found in y_true, recall is set t

## Train MLP-Mixer

In [ ]:
mlp_mixer = create_mlp_mixer(channels, num_classes)
global_client_mlp_mixer = GlobalClient(
    model=mlp_mixer,
    lmdb_path="data/BigEarth_Serbia_Summer_S2.lmdb",
    val_path="data/test.csv",
    csv_paths=["data/c1_train.csv", "data/c2_train.csv", "data/c3_train.csv"],
)

In [ ]:
global_mlp_mixer_results, global_mlp_mixer_client_results = global_client_mlp_mixer.train(communication_rounds=communication_rounds, epochs=epochs)